En este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉.

In [215]:
import pandas as pd
import numpy as np
import sidetable
import re

In [216]:
df=pd.read_csv('../EDA/datos/attack_limpio.csv', index_col=0)

In [217]:
df.head(2)

,case_number,year,country,age,time,species_,mes,fatal_,sex2
0,1800.00.00,1800,seychelles,NaN,NaN,NaN,NaN,y,F
1,1797.05.28.R,1797,NaN,NaN,NaN,NaN,May,y,NaN


1. Columna de species: 
   
   Si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo.
   - Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías
    que hemos definido previamente.
         
        *imaginemos que el valor de una celda es el siguiente

                    'White shark, 3.5 m'

        *tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 
        Un patrón que podriamos usar es: 

                    patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 

        *de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 

    - Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [218]:
patronblanco= '.*([W|w]hite|HITE).*'

patrontigre= '.*([T|t]iger|IGER).*'

patronbull= '.*([B|b]ull|ULL).*'

patrongrey= '.*([G|g]rey|REY).*'

patronlemon= '.*([L|l]emon|LEMON).*'


In [219]:
df=df.fillna("NAN")

In [220]:
#Pair Lucia Lara Anais 
def tiburon(val):

    if re.findall(patronblanco,val):
        df['cleaned_species']= 'White'
        return val.replace(val,'White')
    elif re.findall(patrontigre,val):
        df['cleaned_species']= 'Tiger'
        return val.replace(val,'Tiger')  
    elif re.findall(patronbull,val):
        df['cleaned_species']= 'Bull'
        return val.replace(val,'Bull')
    elif re.findall(patrongrey,val):
        df['cleaned_species']= 'Grey'
        return val.replace(val,'Grey')
    elif re.findall(patronlemon,val):
        df['cleaned_species']= 'Lemon'
        return val.replace(val,'Lemon')
    else:
         return val.replace(val,'Undefined') 
    

In [221]:
df['cleaned_species']=df['species_'].apply(tiburon)

In [222]:
df.sample(10)

,case_number,year,country,age,time,species_,mes,fatal_,sex2,cleaned_species
2162,2000.10.09,2000,usa,17,Morning,2.1 to 2.4 m [7' to 8'] shark,Oct,n,M,Undefined
5546,1911.03.29.R,1911,australia,NAN,NAN,NAN,Mar,y,M,Undefined
432,2016.06.14,2016,usa,6,17h30,3' to 4' shark,Jun,n,F,Undefined
1832,2004.05.23.a,2004,usa,9,12h30,NAN,May,n,M,Undefined
2621,1994.01.03,1994,south africa,33,16h15,"1.3 m [4'3""] shark",Jan,n,M,Undefined
2956,1986.12.22,1986,south africa,21,17h00,3 m [10'] white shark,Dec,y,M,White
939,2012.06.26.b,2012,usa,26,15h00,NAN,Jun,n,M,Undefined
5355,1923.10.20,1923,australia,NAN,NAN,Shark involvement prior to death unconfirmed,Oct,NAN,M,Undefined
3477,1972.12.08,1972,australia,24,NAN,NAN,Dec,n,NAN,Undefined
3478,1972.12.01,1972,reunion,NAN,NAN,NAN,Dec,y,NAN,Undefined


2. Columna de age: 
   
   es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
   - Edad en formato string
   - Edades separadas por &, or, to, >
   - Edades con ?
   Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:
   - Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
   - Puede que os salga un error similar a este:
  
                     TypeError: expected string or bytes-like object
                  
     Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:

                     df['nombre_columna'] = df['nombre_columna'].astype(str)
                     
   - Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. 
   - Por último cambiad el tipo de la columna de string a integer.
  


In [223]:
df['age']=df['age'].astype(str)


In [224]:
edades=list(df['age'].unique())
print(edades)


['NAN', 'young', '14', '19', '20', '15', '21', '27', '36', '25', '6', '16', '50', '13 or 14', '57', '11', '48', '18', '52', '12', '32', '10', '34', '30', '60', '33', '29', '54', '41', '37', '56', '69', '38', '55', '35', '46', '45', '40s', '28', '24', '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8', '82', '73', '20s', '68', '51', '39', '58', 'Teen', '47', '61', '65', '66', '43', '60s', '9', '72', '59', '64', '23', '71', '44', '62', '63', '70', '18 months', '53', '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20', '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26', '8 or 10', '84', '\xa0 ', ' ', '30 or 36', '6½', '21 & ?', '75', '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28', '20?', "60's", '32 & 30', '16 to 18', '87', '67', 'Elderly', 'mid-20s', 'Ca. 33', '74 ', '45 ', '21 or 26', '20 ', '>50', '18 to 22', 'adult', '9 & 12', '? & 19', '9 months', '25 to 35', '23 & 26', '1', '(adult)', '33 & 37', '25 or 28', '37, 67, 35, 27,  ? & 27',

In [225]:

def edad(x):
    try:
        datos=re.findall('\d+',x)
        return datos
    except:
        pass
        


In [226]:
df['age']=df['age'].apply(edad)

In [227]:
df.sample(10)

,case_number,year,country,age,time,species_,mes,fatal_,sex2,cleaned_species
1184,2010.06.27,2010,usa,[20],NAN,"Bull shark, 5'",Jun,n,M,Bull
2931,1987.08.20,1987,usa,[12],11h30,1.2 m to 1.5 m [4' to 5'] shark,Aug,n,M,Undefined
3931,1963.01.04,1963,australia,[],NAN,"White shark, 16', 2,312-lb",Jan,n,NAN,White
3716,1966.08.27.b,1966,south africa,[],NAN,Invalid,Aug,NAN,M,Undefined
6096,1864.07.25,1864,spain,[],Evening,NAN,Jul,y,M,Undefined
2294,1999.03.14.b,1999,new caledonia,[],Morning,200 to 300 kg shark,Mar,n,M,Undefined
2753,1991.07.07,1991,usa,[22],18h45,NAN,Jul,n,M,Undefined
4203,1959.11.22,1959,australia,[19],11h00,3 m to 4.3 m [10' to 14'] shark,Nov,n,M,Undefined
3576,1969.11.30,1969,australia,[15],Afternoon,NAN,Nov,n,M,Undefined
4224,1959.08.30,1959,australia,[],NAN,Invalid,Aug,NAN,M,Undefined


In [228]:
df=df.explode(["age"], ignore_index=False)

In [229]:
df.sample(10)

,case_number,year,country,age,time,species_,mes,fatal_,sex2,cleaned_species
4579,1952.00.00.b,1952,papua new guinea,27,NAN,NAN,NAN,y,M,Undefined
3863,1964.01.27,1964,fiji,NaN,NAN,NAN,Jan,y,NAN,Undefined
2730,1992.01.23,1992,south africa,19,17h30,"Zambesi shark, 1.7 m [5.5']",Jan,n,M,Undefined
1786,2004.11.15,2004,south africa,77,07h00,6 m [20'] white shark,Nov,y,F,White
4587,1951.11.28.a,1951,south africa,24,17h15,"White shark, 2.4 m [8']",Nov,n,M,White
4069,1961.04.00,1961,australia,NaN,NAN,"Wobbegong shark, 1.4 m [4.6']",Apr,n,M,Undefined
4974,1938.06.08,1938,australia,NaN,NAN,6' shark,Jun,n,M,Undefined
280,2017.07.26.R,2017,bahamas,NaN,NAN,NAN,Jul,n,M,Undefined
2921,1987.12.17,1987,usa,16,13h00,1.8 m [6'] blacktip shark,Dec,n,M,Undefined
610,2015.03.18,2015,usa,58,11h45,"Tiger shark, 8 to 12 feet",Mar,n,M,Tiger


In [230]:
df.drop_duplicates(inplace=True)

In [231]:
df.describe()

,year
count,6327.000000
mean,1927.127865
std,281.576548
min,0.000000
25%,1942.000000
50%,1976.000000
75%,2005.000000
max,2018.000000


In [232]:
df["age"]=df["age"].astype(float)

In [236]:
media=round(np.mean(df["age"]))
media

27

In [237]:
#Decidimos reemplazar los nulos por la media de la columna 
df["age"]=df["age"].fillna(media)

In [238]:
df["age"]=df["age"].astype(int)

In [239]:
df.dtypes

case_number        object
year                int64
country            object
age                 int64
time               object
species_           object
mes                object
fatal_             object
sex2               object
cleaned_species    object
dtype: object

In [244]:
df.sample(10)

,case_number,year,country,age,time,species_,mes,fatal_,sex2,cleaned_species
1799,2004.10.01,2004,usa,40,14h00,NAN,Oct,n,M,Undefined
4546,1953.02.00,1953,australia,27,NAN,Wobbegong shark,Feb,n,M,Undefined
2109,2001.05.18,2001,philippines,24,NAN,NAN,May,n,M,Undefined
3895,1963.10.15,1963,north pacific ocean,20,NAN,NAN,Oct,n,M,Undefined
5824,1893.04.04,1893,australia,27,16h30,Shark involvement prior to death unconfirmed,Apr,NAN,M,Undefined
135,ND.0036,0,saudi arabia,27,NAN,6' shark,NAN,n,M,Undefined
4968,1938.08.18,1938,usa,27,NAN,NAN,Aug,n,M,Undefined
402,2016.08.04,2016,usa,22,NAN,Blacktip shark,Aug,n,M,Undefined
1503,2007.08.28.b,2007,usa,15,16h30,"Tiger shark, 10' to 12'",Aug,n,M,Tiger
2643,1993.08.15.b,1993,usa,27,11h30,">6 m white shark, according to witnesses",Aug,n,NAN,White


3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [245]:
df.to_csv("../EDA/datos/attack_limpio.csv")

In [247]:
df2=pd.read_csv('../EDA/datos/attack_limpio.csv', index_col=0)
df2.head(2)

,case_number,year,country,age,time,species_,mes,fatal_,sex2,cleaned_species
0,1800.00.00,1800,seychelles,27,NAN,NAN,NAN,y,F,Undefined
1,1797.05.28.R,1797,NAN,27,NAN,NAN,May,y,NAN,Undefined
